# Pipeline: spaCy → chunking → Ollama walidacja hintów → placeholdery
Przykład end-to-end: surowy tekst → preprocess (spaCy) → jeden chunk na hint → klasyfikacja w Ollama → zamiana na placeholdery.


## 0. (opcjonalnie) instalacja
Zakładamy, że środowisko ma `spacy`, model `pl_core_news_md` oraz `ollama`. W razie potrzeby:
```bash
pip install -r requirements.txt
python -m spacy download pl_core_news_md
```


In [16]:
from spacy_processing import SpacyPreprocessor
from context_manager import ContextManager
from ollama_classifier import OllamaEntityClassifier
from classes import EntityHint
from format import apply_placeholders


## 1. Surowy tekst + wstępne hinty (z innego modułu/rule-based)


In [17]:
raw_text = (
    "Reprezentujemy konsorcjum DataSafe, które zajmuje się anonimizacją dokumentów. "
    "Siedziba firmy znajduje się we Wrocławiu przy ulicy Kościuszki 10. "
    "Dane osobowe takich osób jak Jan Kowalski czy Anna Nowak muszą zostać zanonimizowane. "
    "Mój nr telefonu to 123-456-789. Nazywam się Krawiec i urodziłem się 20-10-2024."
)

hints = [
    EntityHint(text='Wrocławiu', label='placeName', start_char=110, end_char=119),
    EntityHint(text='ulicy', label='geogName', start_char=125, end_char=130),
    EntityHint(text='Jan Kowalski', label='persName', start_char=175, end_char=187),
    EntityHint(text='Anna Nowak', label='persName', start_char=192, end_char=202),
    EntityHint(text='Krawiec', label='persName', start_char=276, end_char=283),
]
hints


[EntityHint(text='Wrocławiu', label='placeName', start_char=110, end_char=119),
 EntityHint(text='ulicy', label='geogName', start_char=125, end_char=130),
 EntityHint(text='Jan Kowalski', label='persName', start_char=175, end_char=187),
 EntityHint(text='Anna Nowak', label='persName', start_char=192, end_char=202),
 EntityHint(text='Krawiec', label='persName', start_char=276, end_char=283)]

## 2. Preprocessing spaCy


In [18]:
pre = SpacyPreprocessor()
pre_result = pre(raw_text)
print(pre_result.meta)


{'model_name': 'pl_core_news_md', 'use_ner_hints': True, 'num_tokens': 57, 'num_sentences': 5, 'num_entities': 5}


## 3. Chunk per hint (minimalny kontekst)
`sentence_radius=0` → tylko zdanie z hintem. Zwiększ do 1, by dodać sąsiednie zdania.


In [19]:
cm = ContextManager()
chunks = cm.chunk_by_hints(pre_result, hints=hints, sentence_radius=0)

for i, ch in enumerate(chunks):
    print(f"Chunk {i}: chars {ch.start_char}-{ch.end_char}, sentences={len(ch.sentences)}, entities={len(ch.entities)}")
    print(ch.text)
    print('---')


Chunk 0: chars 79-145, sentences=1, entities=1
Siedziba firmy znajduje się we Wrocławiu przy ulicy Kościuszki 10.
---
Chunk 1: chars 79-145, sentences=1, entities=1
Siedziba firmy znajduje się we Wrocławiu przy ulicy Kościuszki 10.
---
Chunk 2: chars 146-231, sentences=1, entities=1
Dane osobowe takich osób jak Jan Kowalski czy Anna Nowak muszą zostać zanonimizowane.
---
Chunk 3: chars 146-231, sentences=1, entities=1
Dane osobowe takich osób jak Jan Kowalski czy Anna Nowak muszą zostać zanonimizowane.
---
Chunk 4: chars 264-311, sentences=1, entities=1
Nazywam się Krawiec i urodziłem się 20-10-2024.
---


## 4. Klasyfikacja w Ollama (JSON-only)
Model zwraca etykietę z listy albo `none` (odrzuca hint).


In [20]:
classifier = OllamaEntityClassifier()

try:
    classified = classifier.classify_document(chunks)
except Exception as exc:
    print("Nie udało się wywołać Ollama (uruchom serwer na localhost:11434):", exc)
else:
    for (start, end), label in sorted(classified.items()):
        span = raw_text[start:end]
        status = 'OK' if label != 'none' else 'ODRZUCONE'
        print(f"{status:10} {label:18} -> {span!r}")



Tekst dokumentu (fragment):

"""Siedziba firmy znajduje się we Wrocławiu przy ulicy Kościuszki 10."""

Lista kandydatów do klasyfikacji (id, fragment, wstępna etykieta, pozycja w TEKŚCIE FRAGMENTU):
[
  {
    "id": 0,
    "text": "Wrocławiu",
    "hint_label": "city",
    "start": 31,
    "end": 40
  }
]

Dla KAŻDEGO kandydata wybierz JEDNĄ etykietę z listy (lub "none" jeśli nie pasuje):

- name
- surname
- age
- date-of-birth
- date
- sex
- religion
- political-view
- ethnicity
- sexual-orientation
- health
- relative
- city
- address
- email
- phone
- pesel
- document-number
- company
- school-name
- job-title
- bank-account
- credit-card-number
- username
- secret
- none

Zwróć WYŁĄCZNIE JSON w formacie:

{
  "entities": [
    {"id": <liczba>, "label": "<etykieta>"},
    ...
  ]
}

Nie udało się wywołać Ollama (uruchom serwer na localhost:11434): Ollama chat failed (host=http://localhost:11434, model=SpeakLeash/bielik-7b-instruct-v0.1-gguf:Q4_K_S)


## 5. (opcjonalnie) Zamiana na placeholdery
Przydaje się do anonimizacji tekstu na podstawie zatwierdzonych etykiet.


In [21]:
if 'classified' in locals():
    redacted = apply_placeholders(raw_text, classified)
    print(redacted)


In [ ]:
import json
from dataclasses import dataclass
from typing import Dict

import ollama


# -----------------------------
# 1. System prompt (jeden, ale krótki)
# -----------------------------
ENTITY_SYSTEM_PROMPT = """
Jesteś klasyfikatorem danych osobowych w systemie anonimizacji dokumentów.

Dostajesz:
- fragment tekstu (po polsku),
- jedną lub więcej encji-kandydatów z polami: id, text, hint_label, start, end.

Twoje zadanie:
- dla KAŻDEGO kandydata wybrać DOKŁADNIE jedną etykietę z listy
  albo "none", jeśli fragment na pewno nie jest daną osobową lub wrażliwą.

Dostępne etykiety:
- name
- surname
- age
- date-of-birth
- date
- sex
- religion
- political-view
- ethnicity
- sexual-orientation
- health
- relative
- city
- address
- email
- phone
- pesel
- document-number
- company
- school-name
- job-title
- bank-account
- credit-card-number
- username
- secret
- none

Zasady:
- każdy id z listy wejściowej MUSI pojawić się dokładnie raz w entities.
- jeśli masz wątpliwość, wybierz etykietę bardziej ostrożną (np. health zamiast none przy opisie choroby).
- nie zmieniaj tekstu kandydatów, klasyfikujesz tylko ich typ.

Zwróć WYŁĄCZNIE poprawny JSON o strukturze:
{
  "entities": [
    { "id": <liczba>, "label": "<jedna_etykieta_z_listy_lub_none>" }
  ]
}
"""


# -----------------------------
# 2. Prosty hint (możesz podmienić na swój z classes.py)
# -----------------------------
@dataclass
class EntityHint:
    text: str
    label: str
    start_char: int
    end_char: int


# -----------------------------
# 3. Klasyfikator jednego zdania / jednego hinta
# -----------------------------
class SingleHintClassifier:
    def __init__(
        self,
        model: str = "SpeakLeash/bielik-7b-instruct-v0.1-gguf:Q4_K_S",
        base_url: str = "http://localhost:11434",
        timeout: int = 120,
    ) -> None:
        self.model = model
        self.client = ollama.Client(host=base_url.rstrip("/"), timeout=timeout)

    def _build_user_prompt(self, sentence: str, hint: EntityHint) -> str:
        """
        Build minimal user prompt: one sentence + one candidate.
        """
        candidate = {
            "id": 0,
            "text": hint.text,
            "hint_label": hint.label,
            "start": hint.start_char,
            "end": hint.end_char,
        }
        candidates_json = json.dumps([candidate], ensure_ascii=False, indent=2)

        prompt = f"""
Tekst (jedno zdanie lub fragment):
\"\"\"{sentence}\"\"\"

Kandydat do klasyfikacji (JSON):
{candidates_json}

Dla powyższego kandydata wybierz dokładnie jedną etykietę z listy z instrukcji systemowej
i zwróć WYŁĄCZNIE JSON w ustalonym formacie.
"""
        return prompt

    def _call_ollama(self, user_prompt: str) -> Dict:
        """
        Call Ollama /api/chat via Python client and parse JSON.
        """
        resp = self.client.chat(
            model=self.model,
            messages=[
                {"role": "system", "content": ENTITY_SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt},
            ],
            format="json",
            options={
                "temperature": 0.1,
                "top_p": 0.9,
                "num_predict": 256,  # small, we expect short JSON
            },
        )
        message = resp.get("message", {})
        content = message.get("content")
        if not content:
            raise ValueError(f"Ollama returned empty content: {resp}")
        try:
            data = json.loads(content)
        except json.JSONDecodeError as e:
            raise ValueError(f"Ollama returned non-JSON content: {content}") from e
        return data

    def classify_single(self, sentence: str, hint: EntityHint) -> str:
        """
        Classify a single hint inside a single sentence.
        Returns label string, e.g. 'name', 'city', 'health', or 'none'.
        """
        user_prompt = self._build_user_prompt(sentence, hint)
        data = self._call_ollama(user_prompt)

        entities = data.get("entities", [])
        if not entities:
            raise ValueError(f"No 'entities' in response: {data}")

        # We expect exactly one entity with id 0
        ent = entities[0]
        if ent.get("id") != 0:
            # Fallback: look for id==0 explicitly
            by_id = {e.get("id"): e for e in entities}
            ent = by_id.get(0, ent)

        label = ent.get("label")
        if label is None:
            raise ValueError(f"No label found for id=0 in response: {data}")
        return label


# -----------------------------
# 4. Example usage
# -----------------------------
if __name__ == "__main__":
    # Example sentence from your description
    sentence = (
        "Mój nr telefonu to 123-456-789. Nazywam się Krawiec i urodziłem się 20-10-2024."
    )

    pre_result= SpacyPreprocessor(sentence)

    classifier = SingleHintClassifier()
    label = classifier.classify_single(sentence, pre_result.entities)
    print(f"Hint text: {hint.text!r} -> label: {label!r}")

RuntimeError: spaCy model 'Mój nr telefonu to 123-456-789. Nazywam się Krawiec i urodziłem się 20-10-2024.' is not installed. Install with: python -m spacy download Mój nr telefonu to 123-456-789. Nazywam się Krawiec i urodziłem się 20-10-2024.